<h1> <b> Purpose of this analysis</b></h1>
Purpose of this mini project is to experiment some of data wrangling techniques. We have rcieved Lending Club Loan Data set from 2007 to 2015. There are total 2.2 million rows and 145 columns. It is important to do some pre-processing work in order to analyze data and fill out missing values.  

<h1> <b> Packages</b></h1>
We will start by importing some of packages. Following packages will be imported.


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

sns.set_style('whitegrid')

Now we will be reading data from "Loan.csv" to Pandas Dataframe.

In [2]:
data = pd.read_csv("loan.csv", low_memory=False)



Let's quickly check the shape of data.This will give us idea as how large is current dataset. As we can see, current data set contains more than 2 million rows and 145 columns. Not all columns are useful for our analysis. 

In [3]:
# print(list(data.columns))
print(data.shape)

(2260668, 145)


Here  we will be creating Dataframe called "df_null". This Data frame consist of column name with percent of missing data in each column in descending order. It is noted that any column that has more than 60% of data missing are useless for our analysis and we will simply drop it. 
As we see some column like "url", "id", "member_id" has 100% missing values. Apart from that lot of columns have more than 80% missing data. We will simply drop column which has more than 60% data missing. 

In [4]:
# print((data.isna().sum()[data.isna().sum() > 0]))
df_null = pd.DataFrame({'Count': data.isnull().sum(), 'Percent': 100*data.isnull().sum()/len(data)})
print(df_null[df_null['Percent'] > 0].sort_values(by='Percent', ascending=False))


# print(data.head(3))

                                              Count     Percent
id                                          2260668  100.000000
url                                         2260668  100.000000
member_id                                   2260668  100.000000
orig_projected_additional_accrued_interest  2252242   99.627278
hardship_length                             2250055   99.530537
hardship_reason                             2250055   99.530537
hardship_status                             2250055   99.530537
deferral_term                               2250055   99.530537
hardship_amount                             2250055   99.530537
hardship_start_date                         2250055   99.530537
hardship_end_date                           2250055   99.530537
payment_plan_start_date                     2250055   99.530537
hardship_loan_status                        2250055   99.530537
hardship_dpd                                2250055   99.530537
hardship_payoff_balance_amount          

We will create a list of columns that a data missing more than 60%. We will simply drop those columns and create new dataframe called df_clean. We wil print shape of new dataframe to see how much data has been reduced. We can see total count of columns from 145 to reduced to 103. 

In [5]:
# Creating list of column with more than 60% data missing. 
list_60 = list(df_null[df_null['Percent']>60].index)

df_clean = data.drop(list_60, axis=1)
print(df_clean.shape)

(2260668, 103)


Now that we have eliminated major columns that had missing values. Lets analyze remaining column with missing values. For that we will print all column that has missing data and see how we can predict missing values. We will predict all columns that has missing data grether than 10%. Since we already dropped columns that has missing data more than 60%, in our dataframe.

In [6]:
# print(df_clean.isna().sum())
# print(df_clean.info())

df_null = pd.DataFrame({'Count': df_clean.isnull().sum(), 'Percent': 100*df_clean.isnull().sum()/len(df_clean)})
print(df_null[df_null['Percent'] > 10])


                          Count    Percent
mths_since_last_delinq  1158502  51.246003
next_pymnt_d            1303607  57.664681
open_acc_6m              866130  38.313012
open_act_il              866129  38.312968
open_il_12m              866129  38.312968
open_il_24m              866129  38.312968
mths_since_rcnt_il       909924  40.250227
total_bal_il             866129  38.312968
il_util                 1068850  47.280273
open_rv_12m              866129  38.312968
open_rv_24m              866129  38.312968
max_bal_bc               866129  38.312968
all_util                 866348  38.322655
inq_fi                   866129  38.312968
total_cu_tl              866130  38.313012
inq_last_12m             866130  38.313012
mths_since_recent_inq    295435  13.068482


Lets examine few values of this columns so we can get idea what kind of data estimation we need to do.

In [7]:
col_10 = list(df_null[df_null['Percent'] > 10].index)
print(df_clean[col_10].head(3))




   mths_since_last_delinq next_pymnt_d  open_acc_6m  open_act_il  open_il_12m  \
0                     NaN     Mar-2019          2.0          2.0          1.0   
1                    71.0     Mar-2019          4.0          4.0          2.0   
2                     NaN     Mar-2019          0.0          1.0          0.0   

   open_il_24m  mths_since_rcnt_il  total_bal_il  il_util  open_rv_12m  \
0          2.0                 2.0       12560.0     69.0          2.0   
1          3.0                 3.0       87153.0     88.0          4.0   
2          2.0                14.0        7150.0     72.0          0.0   

   open_rv_24m  max_bal_bc  all_util  inq_fi  total_cu_tl  inq_last_12m  \
0          7.0      2137.0      28.0     1.0         11.0           2.0   
1          5.0       998.0      57.0     2.0         15.0           2.0   
2          2.0         0.0      35.0     1.0          5.0           0.0   

   mths_since_recent_inq  
0                    2.0  
1                    4.

As we see above data set most missing data is numeric and we can use mode or median in order to fill missing values. By looking at dataset and name of columns, mode will be better option compared to median for most of columns. For "total_bil_il" we will use median. Lets analyze mode and median of those columns. But before we need to drop 3 columns which is not useful for our analysis. Those are 
1)  "next_pymnt_dt
2) 'last_credit_pull_d'
3) 'last_pymnt_d'

Lets create a list of these columns. 

In [8]:
# print(df_clean.shape)
cols_to_drop = ['next_pymnt_d', 'last_credit_pull_d', 'last_pymnt_d']
df_clean.drop(cols_to_drop, axis=1, inplace=True)
df_null = pd.DataFrame({'Count': df_clean.isnull().sum(), 'Percent': 100*df_clean.isnull().sum()/len(df_clean)})

Now we will eamine mode and median of all columns those we trying to do estiamtion.



In [18]:
# # df_null = pd.DataFrame({'Count': df_clean.isnull().sum(), 'Percent': 100*df_clean.isnull().sum()/len(df_clean)})

# # ##printing columns with null count more than 0
# # # print(df_null)
# df_clean['annual_inc'].fillna(df_clean['annual_inc'].median(), inplace=True)
# df_clean['emp_length'].fillna(df_clean['emp_length'].median(), inplace=True)

# df_clean['emp_title'].fillna('Unknown', inplace=True)
# df_clean['title'].fillna('Unknown', inplace=True)

# cols_0 = ['open_il_12m', 'open_il_24m', 'total_bal_il', 'open_rv_12m', 'open_rv_24m', 'total_cu_tl', \
#         'inq_last_12m', 'inq_fi', 'max_bal_bc', 'total_il_high_credit_limit', 'total_bc_limit', 'total_bal_ex_mort', \
#          'percent_bc_gt_75', 'num_tl_op_past_12m', 'num_tl_90g_dpd_24m', 'num_tl_30dpd', 'num_tl_120dpd_2m', \
#           'num_accts_ever_120_pd', 'mort_acc', 'tot_coll_amt', 'tot_cur_bal', 'avg_cur_bal', 'bc_open_to_buy',\
#           'bc_util', 'inq_last_6mths', 'collections_12_mths_ex_med', 'chargeoff_within_12_mths', 'pub_rec_bankruptcies',\
#          'tax_liens']
# df_clean[cols_0] = df_clean[cols_0].fillna(0)

# cols_mode = ['mths_since_rcnt_il', 'il_util', 'all_util', 'total_rev_hi_lim', 'acc_open_past_24mths',\
#             'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',\
#             'mths_since_recent_bc', 'mths_since_recent_inq', 'num_actv_bc_tl', 'num_actv_rev_tl',\
#             'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts',\
#             'num_rev_tl_bal_gt_0', 'num_sats', 'pct_tl_nvr_dlq', 'tot_hi_cred_lim', 'delinq_2yrs', 'dti', 'revol_util',\
#            'open_acc_6m', 'open_act_il', 'mths_since_last_delinq']
# df_clean[cols_mode] = df_clean[cols_mode].fillna(df_clean.mode().iloc[0])

# # print(df_clean['title'].dtypes)
# df_null = pd.DataFrame({'Count': df_clean.isnull().sum(), 'Percent': 100*df_clean.isnull().sum()/len(df_clean)})
# index_list = list(df_null.index)
col_10 = list(df_null[df_null['Percent'] > 10].index)
# print(col_10)
# print(df_null.loc[col_10, :])
for index in col_10:
    df_null.loc[index, 'mode'] = df_clean[index].mode()[0]
    df_null.loc[index, 'median'] = df_clean[index].median()
print(df_null.loc[col_10, :])



                          Count    Percent  mode   median
mths_since_last_delinq  1158502  51.246003  12.0     31.0
open_acc_6m              866130  38.313012   0.0      1.0
open_act_il              866129  38.312968   1.0      2.0
open_il_12m              866129  38.312968   0.0      0.0
open_il_24m              866129  38.312968   1.0      1.0
mths_since_rcnt_il       909924  40.250227   7.0     13.0
total_bal_il             866129  38.312968   0.0  23127.0
il_util                 1068850  47.280273  78.0     72.0
open_rv_12m              866129  38.312968   0.0      1.0
open_rv_24m              866129  38.312968   1.0      2.0
max_bal_bc               866129  38.312968   0.0   4413.0
all_util                 866348  38.322655  59.0     58.0
inq_fi                   866129  38.312968   0.0      1.0
total_cu_tl              866130  38.313012   0.0      0.0
inq_last_12m             866130  38.313012   0.0      1.0
mths_since_recent_inq    295435  13.068482   1.0      5.0


By looking at above table, it makes more sense to fill all above columns by mode rather than median. Lets fill those missing values in above columns using mode. 

In [19]:
df_clean[col_10] = df_clean[col_10].fillna(df_clean.mode().iloc[0])

Now lets examine remaining missing data. Most columns will have missing data less than 10%. We will simply drop them. 

In [ ]:
# df_clean['annual_inc'].fillna(df_clean['annual_inc'].median(), inplace=True)
# df_clean['emp_length'].fillna(df_clean['emp_length'].median(), inplace=True)

# df_clean['emp_title'].fillna('Unknown', inplace=True)
# df_clean['title'].fillna('Unknown', inplace=True)

# cols_0 = ['open_il_12m', 'open_il_24m', 'total_bal_il', 'open_rv_12m', 'open_rv_24m', 'total_cu_tl', \
#         'inq_last_12m', 'inq_fi', 'max_bal_bc', 'total_il_high_credit_limit', 'total_bc_limit', 'total_bal_ex_mort', \
#          'percent_bc_gt_75', 'num_tl_op_past_12m', 'num_tl_90g_dpd_24m', 'num_tl_30dpd', 'num_tl_120dpd_2m', \
#           'num_accts_ever_120_pd', 'mort_acc', 'tot_coll_amt', 'tot_cur_bal', 'avg_cur_bal', 'bc_open_to_buy',\
#           'bc_util', 'inq_last_6mths', 'collections_12_mths_ex_med', 'chargeoff_within_12_mths', 'pub_rec_bankruptcies',\
#          'tax_liens']
# df_clean[cols_0] = df_clean[cols_0].fillna(0)

# cols_mode = ['mths_since_rcnt_il', 'il_util', 'all_util', 'total_rev_hi_lim', 'acc_open_past_24mths',\
#             'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',\
#             'mths_since_recent_bc', 'mths_since_recent_inq', 'num_actv_bc_tl', 'num_actv_rev_tl',\
#             'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts',\
#             'num_rev_tl_bal_gt_0', 'num_sats', 'pct_tl_nvr_dlq', 'tot_hi_cred_lim', 'delinq_2yrs', 'dti', 'revol_util',\
#            'open_acc_6m', 'open_act_il', 'mths_since_last_delinq']
# df_clean[cols_mode] = df_clean[cols_mode].fillna(df_clean.mode().iloc[0])

# # print(df_clean['title'].dtypes)
# df_null = pd.DataFrame({'Count': df_clean.isnull().sum(), 'Percent': 100*df_clean.isnull().sum()/len(df_clean)})
# index_list = list(df_null.index)

# for index in index_list:
#     df_null.loc[index, 'mode'] = df_clean[index].mode()[0]
# print([df_null[df_null['Percent']>0.2].index])

In [ ]:
# drop_list = ['next_pymnt_d', 'last_credit_pull_d', 'last_pymnt_d']
# df_clean.drop(drop_list, axis=1, inplace=True)

In [ ]:

# # print(df_clean['earliest_cr_line'].mode())
# print(df_clean['inq_last_6mths'].median())
# print(df_clean['emp_length'].value_counts())
# print(df_clean['emp_length'].astype('str').dtypes)
# df_clean['emp_length'] = df_clean['emp_length'].str.replace(r'[+]\s|[a-z]', '')
# df_clean['emp_length'] = df_clean['emp_length'].str.replace('< 1', '0')
# # df_clean['emp_length'] = df_clean['emp_length'].str.replace('10+ ', '10')

# print(df_clean['emp_length'].value_counts(dropna=False))



In [ ]:
# df_clean = df_clean[pd.notnull(df_clean['acc_now_delinq'])]
# df_null = pd.DataFrame({'Count': df_clean.isnull().sum(), 'Percent': 100*df_clean.isnull().sum()/len(df_clean)})
# index_list = list(df_null.index)
# for index in index_list:
#     df_null.loc[index, 'mode'] = df_clean[index].mode()[0]


# print(df_null[df_null['Percent']>0])

In [ ]:
# df_clean.dropna(inplace=True)

In [ ]:
# col_int = ['emp_length', ]
# df_clean[col_int] = df_clean[col_int].astype('int')
# print(df_clean.dtypes)


In [ ]:
print(df_clean.shape)

In [ ]:
print(df_clean.head(3))

In [ ]:
# data['X'].describe().apply("{0:.5f}".format)
pd.options.display.float_format = '{:.2f}'.format
print(df_clean.describe())

In [ ]:
sns.catplot(y = 'int_rate', x = 'grade', data=df_clean, order = sorted(df_clean.grade.unique()) , col='term', kind='box')

In [ ]:
print(sorted(df_clean.grade.unique()))

In [ ]:
sns.catplot( x = 'home_ownership', data = df_clean, kind = 'count', hue='term')

In [ ]:
print(df_clean['home_ownership'].value_counts())

In [ ]:
print(df_clean['loan_status'].value_counts())

In [ ]:
loan_data = df_clean.copy()

In [ ]:
f_loan, ax_loan = plt.subplots(2, 2, figsize=(15, 10))
sns.distplot(loan_data["loan_amnt"], ax=ax_loan[0, 0], color="#F7522F")
sns.violinplot(y=loan_data["loan_amnt"], ax=ax_loan[0, 1], inner="quartile", palette="Reds")
sns.distplot(loan_data["funded_amnt"], ax=ax_loan[1, 0], color="#2F8FF7")
sns.violinplot(y=loan_data["funded_amnt"], ax=ax_loan[1, 1], inner="quartile", palette="Blues")
ax_loan[0, 0].set_title("Loan amount distribution", fontsize=16)
ax_loan[0, 1].set_title("Loan amount distribution", fontsize=16)
ax_loan[1, 0].set_title("Funded amount distribution", fontsize=16)
ax_loan[1, 1].set_title("Funded amount distribution", fontsize=16)
ax_loan[0, 0].set_xlabel("loan amount", fontsize=16)
ax_loan[1, 0].set_xlabel("loan amount", fontsize=16)
ax_loan[0, 1].set_ylabel("loan amount", fontsize=16)
ax_loan[1, 1].set_ylabel("loan amount", fontsize=16)

In [ ]:
loan_data['year'] = pd.to_datetime(loan_data['issue_d']).dt.year
loan_year_num = loan_data['year'].value_counts().to_dict()
loan_year_num_pd = pd.DataFrame(list(loan_year_num.items()), columns=["year", "loan times"])
loan_year_num_pd.sort_values("year", inplace=True)
# print(loan_year_num_pd)
loan_money_count_per_year = loan_data.groupby("year")["loan_amnt"].sum().to_dict()
loan_money_count_per_year_pd = pd.DataFrame(list(loan_money_count_per_year.items()), columns=["year", "loan_amnt"])
loan_money_count_per_year_pd.sort_values("year", inplace=True)
# print(loan_money_count_per_year_pd)

# fig, ax = plt.subplots(1,2, figsize = (15,10))
sns.catplot(x='year', y = 'loan times', data = loan_year_num_pd, palette = 'tab10', kind='bar')
sns.catplot(x='year', y = 'loan_amnt', data = loan_money_count_per_year_pd, palette = 'tab10', kind='bar' )

In [ ]:

loan_amnt_per_state = loan_data[['addr_state', 'loan_amnt']].groupby('addr_state').sum().sort_values(by='loan_amnt', ascending=False).reset_index()
# print(loan_amnt_per_state)
loan_per_state = loan_data['addr_state'].value_counts().rename_axis('addr_state').reset_index(name='counts')
# print(loan_per_state)
fig, (ax1, ax2) = plt.subplots(2,1,figsize=(15,30))
sns.barplot(x='loan_amnt', y = 'addr_state', data = loan_amnt_per_state, orient = 'h', ax=ax1 )
sns.barplot(x='counts', y = 'addr_state', data = loan_per_state, orient = 'h', ax=ax2 )

In [ ]:
# loan_amnt_per_job = loan_data[['emp_title', 'loan_amnt']].groupby('emp_title').mean().sort_values(by = 'loan_amnt', ascending = False).reset_index()
# loan_per_job = loan_data['emp_title'].value_counts().rename_axis('emp_title').reset_index(name = 'counts')
# fig, (ax1, ax2) = plt.subplots(2,1,figsize=(15,30))
# sns.barplot(x='loan_amnt', y = 'emp_title', data = loan_amnt_per_job, orient = 'h', ax=ax1 )
# sns.barplot(x='counts', y = 'emp_title', data = loan_per_job, orient = 'h', ax=ax2 )
